# Example: find websites of National Statistical Institutes (NSIs) 

In [1]:
# Note: you need to have installed the urlfinding module in Anaconda, see Readme.txt
import urlfinding as uf
import pandas as pd

# You can ignore a FutureWarning about sklearn.metrics.classification being deprecated 

C:\Users\OBOS\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Input data:
pd.read_csv('data/NSIs.csv', delimiter = ';').head()

,id,tradename,legalname,address,postalcode,locality,country,phone
0,1,Danmarks Statistik,Danmarks Statistik,"Sejrøgade 11, Postboks 2550",2100,KØBENHAVN Ø,Denmark,(45) 3917 3917
1,2,Statistics Estonia,Statistics Estonia,Tatari 51,10134,Tallinn,Estonia,(372) 6259 300
2,3,"INSEE, Direction générale","INSEE, Direction générale",88 avenue Verdier - CS 70058,92541,Montrouge Cedex.,France,(33) 1 87 69 50 00
3,4,ISTAT,Istituto Nazionale di Statistica,Via Cesare Balbo,184,Roma,Italy,(39) 06 4673 2243-2244
4,5,Statistical Service of Cyprus,Statistical Service of Cyprus,Michalakis Karaolis street,1444,Nicosia,Cyprus,(357) 22 602 102


## Search via Google API:

In [3]:
# Note1: This takes a while (a few minutes in this example).
#        For each record 6 queries are fired with an idle time in between.
#        During execution the search function displays which record is being processed.
# Note2: if you see a 403 error, you reached the maximum number of google queries for your API key.
# Note3: As documented in the API in the readme this search function starts at the next record from the last session. 
#        Hence if you want to start from scratch either remove the maxrownum file or set the number in that file to 0 

base_file    = './data/NSIs.csv'
googleconfig = './config/config.yml'
blacklist    = './data/blacklist.txt'
startrow     = 0                          # optional: if absent the search will continue from the last session
nrows        = 14                         # maximum number of rows to search 

uf.search(base_file, googleconfig, blacklist, startrow, nrows)


At record: 13
Searchresults saved in C:\Users\OBOS\GH\urlfinding\examples/data/20200115searchResult.csv


## Extract features from the search results:

In [4]:
date       = '20200115'
data_files = ['./data/20200115searchResult.csv'] # change this into the file created at the previous step
                                                 # in a multi search session you can use multiple searchResult files here
blacklist  = './data/blacklist.txt'

uf.extract(date, data_files, blacklist)

Created feature file C:\Users\OBOS\GH\urlfinding\examples/data/20200115features_agg.csv


## Predict urls:

In [5]:
# Note: you might get a UserWarning about different versions of pickle estimators, which you can safely ignore.

model_file   = './data/model.pkl'
feature_file = './data/20200115features_agg.csv'
base_file    = './data/NSIs.csv'

uf.predict(feature_file, model_file, base_file)

Predictions saved in ./data/NSIs_url.csv


In [7]:
# All results:
df = pd.read_csv('data/NSIs_url.csv', delimiter = ';')
df[['tradename', 'country', 'host', 'eqPred', 'pTrue']]

# The column host contains the predicted domain. At the moment only two level domains are supported.
# The column eqPred indicates whether the predicted url is the right one.
# The column pTrue shows the confidence of the prediction.

,tradename,country,host,eqPred,pTrue
0,Danmarks Statistik,Denmark,dst.dk,True,0.784047
1,Statistics Estonia,Estonia,stat.ee,True,0.949188
2,"INSEE, Direction générale",France,insee.fr,True,0.857058
3,ISTAT,Italy,istat.it,True,0.837411
4,Statistical Service of Cyprus,Cyprus,gov.cy,True,0.967508
5,National Statistics Office,Malta,gov.mt,True,0.783494
6,Statistik Austria,Austria,gv.at,True,0.376872
7,Statistics Poland,Poland,gov.pl,True,0.903021
8,Statistical Office of the Republic of Slovenia,Slovenia,stat.si,True,0.537509
9,Statistical Office of the Slovak Republic,Slovakia,statistics.sk,True,0.666669


In [8]:
# Search results may vary depending on the state of the internet, search engine index and location from where you search.
# Here are the previously saved search results of a session executed on 20200115 from Statistics Netherlands:
df = pd.read_csv('data/NSIs_url_20200115.csv', delimiter = ';')
df[['tradename', 'country', 'host', 'eqPred', 'pTrue']]


,tradename,country,host,eqPred,pTrue
0,Danmarks Statistik,Denmark,dst.dk,True,0.664405
1,Statistics Estonia,Estonia,stat.ee,True,0.926300
2,"INSEE, Direction générale",France,insee.fr,True,0.866456
3,ISTAT,Italy,istat.it,True,0.825502
4,Statistical Service of Cyprus,Cyprus,gov.cy,True,0.972189
5,National Statistics Office,Malta,gov.mt,True,0.782955
6,Statistik Austria,Austria,gv.at,True,0.418751
7,Statistics Poland,Poland,gov.pl,True,0.918104
8,Statistical Office of the Republic of Slovenia,Slovenia,stat.si,True,0.566315
9,Statistical Office of the Slovak Republic,Slovakia,statistics.sk,True,0.659360


## Train a model: